In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
### IMPORTING FILES FROM THE PROJECT ROOT.

import numpy as np
import plotly.graph_objects as go

#### --- MAKING ROOT PROJECT FOLDER VISIBLE AT SCRIPT LEVEL ---

##
## --- Finding root project folder ---
##     (needed to import settings)
##
import os, sys
def find_project_root(beacon="ROOT_BEACON"):
    current_path = os.getcwd()
    
    while True:
        if os.path.isfile(os.path.join(current_path, beacon)):
            return current_path
        
        parent_path = os.path.dirname(current_path)
        if parent_path == current_path:
            raise RuntimeError(f"Unable to find project root.\nNo '{beacon}' file found in any parent directory.")
        
        current_path = parent_path

# --- Find and set project root ---
project_root = find_project_root()


## --- Adding root porject folder to current system path ---
os.chdir(project_root)
    
####                    --- end ---


#### IMPORTING FROM ROOT PROJECT FOLDER 
from Utils import ArraySettings, SortingSettings
from Utils import ArrayDataManager
from Utils import ArrayStorageCompressor

## --- Updating the execution times storage folder ---
if project_root not in SortingSettings.EXECUTION_TIMES_FOLDER:
    SortingSettings.EXECUTION_TIMES_FOLDER = os.sep.join([project_root, SortingSettings.EXECUTION_TIMES_FOLDER])
    SortingSettings.EXECUTION_TIMES_GRAPH_FOLDER = os.sep.join([project_root, SortingSettings.EXECUTION_TIMES_GRAPH_FOLDER])

    assert os.path.exists(SortingSettings.EXECUTION_TIMES_FOLDER), f"Execution times folder does not exists. \nCurrent location set: {SortingSettings.EXECUTION_TIMES_FOLDER}"

    EXECUTION_TIMES_GRAPH_FOLDER_LOGARITMIC = os.sep.join([SortingSettings.EXECUTION_TIMES_GRAPH_FOLDER, "logaritmic"])
    EXECUTION_TIMES_GRAPH_FOLDER_LINEAR = os.sep.join([SortingSettings.EXECUTION_TIMES_GRAPH_FOLDER, "linear"])
    
    if not os.path.exists(EXECUTION_TIMES_GRAPH_FOLDER_LOGARITMIC):
        os.makedirs(EXECUTION_TIMES_GRAPH_FOLDER_LOGARITMIC)

    if not os.path.exists(EXECUTION_TIMES_GRAPH_FOLDER_LINEAR):
        os.makedirs(EXECUTION_TIMES_GRAPH_FOLDER_LINEAR)
    


### LOADING DATA FROM TIME FOLDER

In [ ]:
arrayTimeManager = ArrayDataManager.ExecutionTimeDataStorage()



for file in os.listdir(SortingSettings.EXECUTION_TIMES_FOLDER):
    if SortingSettings.EXECUTION_TIMES_FILE_EXTENSION in file:
        
        full_path_file = os.path.join(SortingSettings.EXECUTION_TIMES_FOLDER, file)
        algorithm_name = file[0:file.find(SortingSettings.EXECUTION_TIMES_FILE_EXTENSION)]
        read_execution_time_storage = ArrayStorageCompressor.readFromFile(full_path_file)
        assert len(read_execution_time_storage.keys()) == 1, f"Expected only one algorithm key in file {full_path_file}, got {read_execution_time_storage.keys()}"
        assert algorithm_name in read_execution_time_storage, f"Expected {algorithm_name} as key in file {full_path_file}, got {read_execution_time_storage.keys()}"

        arrayTimeManager.merge(read_execution_time_storage)
        


## DEFINITION OF FUNCTIONS USED TO SHOW TIME DATA

### DEFINITION OF FUNCTION USED TO COMPUTE THE PLOTTING DATA

In [ ]:
def computePlotData(algorithmName):
    data = {}
    for storage_path, execution_times in arrayTimeManager[algorithmName].items():
        for execution_time in execution_times:
            variability_value = execution_time.get_variability()
            if variability_value not in data:
                data[variability_value] = []
            data[variability_value].extend(
            execution_time.sample)
            
    x = []
    y = []
    e = []
    for variability, execution_times in data.items():
        x.append(variability)
        y.append(np.mean(execution_times))
        sd   = np.array(execution_times).std(ddof=1)
        sem  = sd / np.sqrt(len(execution_times))
        ci95 = 1.96 * sem 
        e.append(ci95)

    return go.Scatter(
                x = x,
                y = y,
                error_y = dict(
                    type = 'data',
                    symmetric=True,
                    array = e,
                    visible = True),
                name = algorithmName,
                marker=dict(size=6, symbol="circle", line=dict(width=1, color="DarkSlateGrey")),
                mode = "lines+markers",
                hovertemplate =
                                '%{y:.4s}s ± %{error_y.array:.4s}s' +
                                '' ## 'extra' keyword removes trace category   
                )

### DEFINITION OF THE FUNCTION USED TO CREATE THE GRAPH LAYOUT

In [ ]:
def createLayout(log= False):

    axis_type = "linear" if not log else "log"
    
    return go.Layout(
        height = 700,
        title = dict(
            text = "Execution Times",
            font = dict(
                size= 30
            ),
            subtitle=dict(
                text = "Run times grouped by sorting algorithm showed in a {type}-{type} graph.".format(type= axis_type),
                font = dict(color="dimgrey", size=16)
            )
        ),
    
        xaxis = dict(
            title = dict(
                text = (ArraySettings.VARIABILITY.value["nice_name"]) + " <i>{type}</i>".format(type= "n" if not log else "log(n)"),
                font=dict(size = 18)
            ),
            type = axis_type,
            hoverformat = ".5s"
        ),
        
        yaxis = dict(
            title = dict(
                text = "Execution time <i>{type}</i>".format(type= "s" if not log else "log(s)"),
                font=dict(size = 18)
            ),
            type = axis_type,
        ),
        
        legend = dict(
            title = dict(
                text = "Algorithms"
            ),
            font=dict(size = 14),
            groupclick='toggleitem',
            itemdoubleclick='toggleothers'
        ),
        
        font=dict(
            family="Georgia",
            color = "black",
            size=18,
        ),
        
        hoverlabel = dict(
            font = dict(
                size = 14,
                family = "monospace"
            )
        ),
        
        hovermode="x unified",

        meta = dict(
            variability= ArraySettings.VARIABILITY.value["nice_name"],
            scale= "logaritmic" if log else "linear"
        )
    )

### PLOT EXECUTION POINTS GROUPED BY SORTING ALGORITHM

### COLLECTING TIMING DATA FROM TIME MANAGER

In [ ]:
data = {}
layout = createLayout(log = False)
fig = go.Figure(layout = layout)
for algorithmName in list(arrayTimeManager.keys()):
    trace = computePlotData(algorithmName)
    data[algorithmName] = trace
    fig.add_trace(trace)
        
fig.show()

### WRITE ON FILE

In [ ]:
algorithmNames = list(arrayTimeManager.keys())

for log in [False, True]:
    destination_folder = EXECUTION_TIMES_GRAPH_FOLDER_LINEAR if not log else EXECUTION_TIMES_GRAPH_FOLDER_LOGARITMIC
    os.makedirs(destination_folder, exist_ok= True)
    layout = createLayout(log = log)
    fig = go.Figure(layout= layout)
    for alg, trace in data.items():
        fig.add_trace(trace)
    file_name = "figure.fig"
    ArrayStorageCompressor.writeOnFile(fig, os.path.join(destination_folder, file_name))